# SCL Package - QR Code Generator
## &emsp; &emsp; &emsp; &emsp; &emsp; &emsp; - Vishnu Na. (20PT39)

### Importing the libraries

In [31]:
from __future__ import annotations
import collections, itertools, re
from collections.abc import Sequence
from typing import Callable, Dict, List, Optional, Tuple, Union

### Functions for QR code generator

In [51]:
class QrCode:
    
    """Instances of this class represent an immutable square grid of dark and light cells.
    The class provides static factory functions to create a QR Code from text or binary data.
    The class covers the QR Code Model 2 specification, supporting all versions (sizes)
    from 1 to 40, all 4 error correction levels, and 4 character encoding modes."""
    
    """Ways to create a QR Code object:
    - High level: Take the payload data and call QrCode.encode_text() or QrCode.encode_binary().
    - Mid level: Custom-make the list of segments and call QrCode.encode_segments().
    - Low level: Custom-make the array of data codeword bytes
                 (including segment headers and final padding, excluding error correction codewords), 
                  supply the appropriate version number, and call the QrCode() constructor."""
    
    def encode_text(text: str, ecl: QrCode.Ecc) -> QrCode:
        #Returns a QR Code representing the given Unicode text string at the given error correction level.
        #Upper bound is 738 code points.
        segs: List[QrSegment] = QrSegment.make_segments(text)
        return QrCode.encode_segments(segs, ecl)

    def encode_binary(data: Union[bytes,Sequence[int]], ecl: QrCode.Ecc) -> QrCode:
        #Returns a QR Code representing the given binary data at the given error correction level.
        #Upper bound is 2953 bytes.
        return QrCode.encode_segments([QrSegment.make_bytes(data)], ecl)

    def encode_segments(segs: Sequence[QrSegment], ecl: QrCode.Ecc, minversion: int = 1, maxversion: int = 40, mask: int = -1, boostecl: bool = True) -> QrCode:
        #Returns a QR Code representing the given segments with the given encoding parameters.
        if not (QrCode.MIN_VERSION <= minversion <= maxversion <= QrCode.MAX_VERSION) or not (-1 <= mask <= 7):
            raise ValueError("Invalid value")

        #Find the minimal version number to use
        for version in range(minversion, maxversion + 1):
            datacapacitybits: int = QrCode._get_num_data_codewords(version, ecl) * 8  #Number of data bits available
            datausedbits: Optional[int] = QrSegment.get_total_bits(segs, version)
            if (datausedbits is not None) and (datausedbits <= datacapacitybits):
                break  #This version number is found to be suitable
            if version >= maxversion:  #All versions in the range could not fit the given data
                msg: str = "Segment too long"
                if datausedbits is not None:
                    msg = f"Data length = {datausedbits} bits, Max capacity = {datacapacitybits} bits"
                raise DataTooLongError(msg)
        assert datausedbits is not None

        #Increase the error correction level while the data still fits in the current version number
        for newecl in (QrCode.Ecc.MEDIUM, QrCode.Ecc.QUARTILE, QrCode.Ecc.HIGH):  # From low to high
            if boostecl and (datausedbits <= QrCode._get_num_data_codewords(version, newecl) * 8):
                ecl = newecl

        #Concatenate all segments to create the data bit string
        bb = _BitBuffer()
        for seg in segs:
            bb.append_bits(seg.get_mode().get_mode_bits(), 4)
            bb.append_bits(seg.get_num_chars(), seg.get_mode().num_char_count_bits(version))
            bb.extend(seg._bitdata)
        assert len(bb) == datausedbits

        #Add terminator and pad up to a byte if applicable
        datacapacitybits = QrCode._get_num_data_codewords(version, ecl) * 8
        assert len(bb) <= datacapacitybits
        bb.append_bits(0, min(4, datacapacitybits - len(bb)))
        bb.append_bits(0, -len(bb) % 8)  # Note: Python's modulo on negative numbers behaves better than C family languages
        assert len(bb) % 8 == 0

        #Pad with alternating bytes until data capacity is reached
        for padbyte in itertools.cycle((0xEC, 0x11)):
            if len(bb) >= datacapacitybits:
                break
            bb.append_bits(padbyte, 8)

        #Pack bits into bytes in big endian
        datacodewords = bytearray([0] * (len(bb) // 8))
        for (i, bit) in enumerate(bb):
            datacodewords[i >> 3] |= bit << (7 - (i & 7))

        #Create the QR Code object
        return QrCode(version, ecl, datacodewords, mask)

    #Version number of QR code (ranges from 1 to 40)
    _version: int

    #The dimension of the QR code (ranges from 21 to 177) (size = 4*version + 17)
    _size: int

    #The error correction level used in this QR Code.
    _errcorlvl: QrCode.Ecc

    #The index of the mask pattern used in this QR Code, which is between 0 and 7.
    _mask: int

    #The modules of this QR Code (White for 0 and Black for 1).
    #Immutable after constructor finishes. Accessed through get_module().
    _modules: List[List[bool]]

    #Indicates function modules that are not subjected to masking. Discarded when constructor finishes.
    _isfunction: List[List[bool]]

    #Constructor
    def __init__(self, version: int, errcorlvl: QrCode.Ecc, datacodewords: Union[bytes,Sequence[int]], msk: int) -> None:
        #Creates a new QR Code with the given version number, error correction level, data codeword bytes, and mask number.
        
        #Check scalar arguments and set fields
        if not (QrCode.MIN_VERSION <= version <= QrCode.MAX_VERSION):
            raise ValueError("Version value out of range")
        if not (-1 <= msk <= 7):
            raise ValueError("Mask value out of range")

        self._version = version
        self._size = version * 4 + 17
        self._errcorlvl = errcorlvl

        #Initialize both grids to be size*size arrays of Boolean false
        self._modules    = [[False] * self._size for _ in range(self._size)]  # Initially all light
        self._isfunction = [[False] * self._size for _ in range(self._size)]

        #Compute ECC, draw modules
        self._draw_function_patterns()
        allcodewords: bytes = self._add_ecc_and_interleave(bytearray(datacodewords))
        self._draw_codewords(allcodewords)

        #Do masking
        if msk == -1:  #Automatically choose best mask
            minpenalty: int = 1 << 32
            for i in range(8):
                self._apply_mask(i)
                self._draw_format_bits(i)
                penalty = self._get_penalty_score()
                if penalty < minpenalty:
                    msk = i
                    minpenalty = penalty
                self._apply_mask(i)  #Undoes the mask due to XOR
        assert 0 <= msk <= 7
        self._mask = msk
        self._apply_mask(msk)  #Apply the final choice of mask
        self._draw_format_bits(msk)  #Overwrite old format bits

        del self._isfunction

    def get_version(self) -> int:
        return self._version

    def get_size(self) -> int:
        return self._size

    def get_error_correction_level(self) -> QrCode.Ecc:
        return self._errcorlvl

    def get_mask(self) -> int:
        return self._mask

    def get_module(self, x: int, y: int) -> bool:
        return (0 <= x < self._size) and (0 <= y < self._size) and self._modules[y][x]

    #Reads this object's version field, and draws and marks all function modules.
    def _draw_function_patterns(self) -> None:
        #Draw horizontal and vertical timing patterns
        for i in range(self._size):
            self._set_function_module(6, i, i % 2 == 0)
            self._set_function_module(i, 6, i % 2 == 0)

        #Draw 3 finder patterns (all corners except bottom right; overwrites some timing modules)
        self._draw_finder_pattern(3, 3)
        self._draw_finder_pattern(self._size - 4, 3)
        self._draw_finder_pattern(3, self._size - 4)

        #Draw numerous alignment patterns
        alignpatpos: List[int] = self._get_alignment_pattern_positions()
        numalign: int = len(alignpatpos)
        skips: Sequence[Tuple[int,int]] = ((0, 0), (0, numalign - 1), (numalign - 1, 0))
        for i in range(numalign):
            for j in range(numalign):
                if (i, j) not in skips:  # Don't draw on the three finder corners
                    self._draw_alignment_pattern(alignpatpos[i], alignpatpos[j])

        #Draw configuration data
        self._draw_format_bits(0)  # Dummy mask value; overwritten later in the constructor
        self._draw_version()


    #Draws two copies of the format bits (with its error correction code)
    def _draw_format_bits(self, mask: int) -> None:
        # Calculate error correction code and pack bits
        data: int = self._errcorlvl.formatbits << 3 | mask  # errCorrLvl is uint2, mask is uint3
        rem: int = data
        for _ in range(10):
            rem = (rem << 1) ^ ((rem >> 9) * 0x537)
        bits: int = (data << 10 | rem) ^ 0x5412  # uint15
        assert bits >> 15 == 0

        #Draw first copy
        for i in range(0, 6):
            self._set_function_module(8, i, _get_bit(bits, i))
        self._set_function_module(8, 7, _get_bit(bits, 6))
        self._set_function_module(8, 8, _get_bit(bits, 7))
        self._set_function_module(7, 8, _get_bit(bits, 8))
        for i in range(9, 15):
            self._set_function_module(14 - i, 8, _get_bit(bits, i))

        #Draw second copy
        for i in range(0, 8):
            self._set_function_module(self._size - 1 - i, 8, _get_bit(bits, i))
        for i in range(8, 15):
            self._set_function_module(8, self._size - 15 + i, _get_bit(bits, i))
        self._set_function_module(8, self._size - 8, True)  # Always dark

    #Draws two copies of the version bits (with its own error correction code)
    def _draw_version(self) -> None:
        if self._version < 7:
            return

        # Calculate error correction code and pack bits
        rem: int = self._version  # version is uint6, in the range [7, 40]
        for _ in range(12):
            rem = (rem << 1) ^ ((rem >> 11) * 0x1F25)
        bits: int = self._version << 12 | rem  # uint18
        assert bits >> 18 == 0

        # Draw two copies
        for i in range(18):
            bit: bool = _get_bit(bits, i)
            a: int = self._size - 11 + i % 3
            b: int = i // 3
            self._set_function_module(a, b, bit)
            self._set_function_module(b, a, bit)

    #Draws a 9*9 finder pattern including the border separator
    def _draw_finder_pattern(self, x: int, y: int) -> None:
        for dy in range(-4, 5):
            for dx in range(-4, 5):
                xx, yy = x + dx, y + dy
                if (0 <= xx < self._size) and (0 <= yy < self._size):
                    # Chebyshev/infinity norm
                    self._set_function_module(xx, yy, max(abs(dx), abs(dy)) not in (2, 4))


    #Draws a 5*5 alignment pattern, with the center module at (x, y).
    def _draw_alignment_pattern(self, x: int, y: int) -> None:
        for dy in range(-2, 3):
            for dx in range(-2, 3):
                self._set_function_module(x + dx, y + dy, max(abs(dx), abs(dy)) != 1)


    def _set_function_module(self, x: int, y: int, isdark: bool) -> None:
        assert type(isdark) is bool
        self._modules[y][x] = isdark
        self._isfunction[y][x] = True

    #Returns a new byte string representing the given data with the appropriate error correction codewords appended to it
    def _add_ecc_and_interleave(self, data: bytearray) -> bytes:
        version: int = self._version
        assert len(data) == QrCode._get_num_data_codewords(version, self._errcorlvl)

        #Calculate parameter numbers
        numblocks: int = QrCode._NUM_ERROR_CORRECTION_BLOCKS[self._errcorlvl.ordinal][version]
        blockecclen: int = QrCode._ECC_CODEWORDS_PER_BLOCK  [self._errcorlvl.ordinal][version]
        rawcodewords: int = QrCode._get_num_raw_data_modules(version) // 8
        numshortblocks: int = numblocks - rawcodewords % numblocks
        shortblocklen: int = rawcodewords // numblocks

        #Split data into blocks and append ECC to each block
        blocks: List[bytes] = []
        rsdiv: bytes = QrCode._reed_solomon_compute_divisor(blockecclen)
        k: int = 0
        for i in range(numblocks):
            dat: bytearray = data[k : k + shortblocklen - blockecclen + (0 if i < numshortblocks else 1)]
            k += len(dat)
            ecc: bytes = QrCode._reed_solomon_compute_remainder(dat, rsdiv)
            if i < numshortblocks:
                dat.append(0)
            blocks.append(dat + ecc)
        assert k == len(data)

        #Interleave (not concatenate) the bytes from every block into a single sequence
        result = bytearray()
        for i in range(len(blocks[0])):
            for (j, blk) in enumerate(blocks):
                # Skip the padding byte in short blocks
                if (i != shortblocklen - blockecclen) or (j >= numshortblocks):
                    result.append(blk[i])
        assert len(result) == rawcodewords
        return result

    #Draws the given sequence of 8-bit codewords (data and error correction) onto the entire data area of this QR Code.
    def _draw_codewords(self, data: bytes) -> None:
        assert len(data) == QrCode._get_num_raw_data_modules(self._version) // 8

        i: int = 0  #Bit index into the data
        #Do the funny zigzag scan
        for right in range(self._size - 1, 0, -2):  #Index of right column in each column pair
            if right <= 6:
                right -= 1
            for vert in range(self._size):  #Vertical counter
                for j in range(2):
                    x: int = right - j  #Actual x coordinate
                    upward: bool = (right + 1) & 2 == 0
                    y: int = (self._size - 1 - vert) if upward else vert  #Actual y coordinate
                    if (not self._isfunction[y][x]) and (i < len(data) * 8):
                        self._modules[y][x] = _get_bit(data[i >> 3], 7 - (i & 7))
                        i += 1
                    #If this QR Code has any remainder bits (0 to 7), they were assigned as
                    #0/false/light by the constructor and are left unchanged by this method
        assert i == len(data) * 8


    #XORs the codeword modules in this QR Code with the given mask pattern.
    def _apply_mask(self, mask: int) -> None:
        if not (0 <= mask <= 7):
            raise ValueError("Mask value out of range")
        masker: Callable[[int,int],int] = QrCode._MASK_PATTERNS[mask]
        for y in range(self._size):
            for x in range(self._size):
                self._modules[y][x] ^= (masker(x, y) == 0) and (not self._isfunction[y][x])


    def _get_penalty_score(self) -> int:
        result: int = 0
        size: int = self._size
        modules: List[List[bool]] = self._modules

        #Adjacent modules in row having same color, and finder-like patterns
        for y in range(size):
            runcolor: bool = False
            runx: int = 0
            runhistory = collections.deque([0] * 7, 7)
            for x in range(size):
                if modules[y][x] == runcolor:
                    runx += 1
                    if runx == 5:
                        result += QrCode._PENALTY_N1
                    elif runx > 5:
                        result += 1
                else:
                    self._finder_penalty_add_history(runx, runhistory)
                    if not runcolor:
                        result += self._finder_penalty_count_patterns(runhistory) * QrCode._PENALTY_N3
                    runcolor = modules[y][x]
                    runx = 1
            result += self._finder_penalty_terminate_and_count(runcolor, runx, runhistory) * QrCode._PENALTY_N3
        #Adjacent modules in column having same color, and finder-like patterns
        for x in range(size):
            runcolor = False
            runy = 0
            runhistory = collections.deque([0] * 7, 7)
            for y in range(size):
                if modules[y][x] == runcolor:
                    runy += 1
                    if runy == 5:
                        result += QrCode._PENALTY_N1
                    elif runy > 5:
                        result += 1
                else:
                    self._finder_penalty_add_history(runy, runhistory)
                    if not runcolor:
                        result += self._finder_penalty_count_patterns(runhistory) * QrCode._PENALTY_N3
                    runcolor = modules[y][x]
                    runy = 1
            result += self._finder_penalty_terminate_and_count(runcolor, runy, runhistory) * QrCode._PENALTY_N3

        #2*2 blocks of modules having same color
        for y in range(size - 1):
            for x in range(size - 1):
                if modules[y][x] == modules[y][x + 1] == modules[y + 1][x] == modules[y + 1][x + 1]:
                    result += QrCode._PENALTY_N2

        #Balance of dark and light modules
        dark: int = sum((1 if cell else 0) for row in modules for cell in row)
        total: int = size**2  #Note that size is odd, so dark/total != 1/2
        #Compute the smallest integer k >= 0 such that (45-5k)% <= dark/total <= (55+5k)%
        k: int = (abs(dark * 20 - total * 10) + total - 1) // total - 1
        assert 0 <= k <= 9
        result += k * QrCode._PENALTY_N4
        assert 0 <= result <= 2568888  #Non-tight upper bound based on default values of PENALTY_N1, ..., N4
        return result

    #Returns an ascending list of positions of alignment patterns for this version number.
    def _get_alignment_pattern_positions(self) -> List[int]:
        ver: int = self._version
        if ver == 1:
            return []
        else:
            numalign: int = ver // 7 + 2
            step: int = 26 if (ver == 32) else \
                (ver * 4 + numalign * 2 + 1) // (numalign * 2 - 2) * 2
            result: List[int] = [(self._size - 7 - i * step) for i in range(numalign - 1)] + [6]
            return list(reversed(result))

    #Returns the number of data bits that can be stored in a QR Code of the given version number, after
    #all function modules are excluded. This includes remainder bits, so it might not be a multiple of 8.
    def _get_num_raw_data_modules(ver: int) -> int:
        if not (QrCode.MIN_VERSION <= ver <= QrCode.MAX_VERSION):
            raise ValueError("Version number out of range")
        result: int = (16 * ver + 128) * ver + 64
        if ver >= 2:
            numalign: int = ver // 7 + 2
            result -= (25 * numalign - 10) * numalign - 55
            if ver >= 7:
                result -= 36
        assert 208 <= result <= 29648
        return result

    #Returns the number of 8-bit data (i.e. not error correction) codewords contained in any
    #QR Code of the given version number and error correction level, with remainder bits discarded.
    def _get_num_data_codewords(ver: int, ecl: QrCode.Ecc) -> int:
        return QrCode._get_num_raw_data_modules(ver) // 8 \
            - QrCode._ECC_CODEWORDS_PER_BLOCK    [ecl.ordinal][ver] \
            * QrCode._NUM_ERROR_CORRECTION_BLOCKS[ecl.ordinal][ver]

    #Returns a Reed-Solomon ECC generator polynomial for the given degree.
    def _reed_solomon_compute_divisor(degree: int) -> bytes:
        if not (1 <= degree <= 255):
            raise ValueError("Degree out of range")
        #Polynomial coefficients are stored from highest to lowest power, excluding the leading term which is always 1.
        #For example the polynomial x^3 + 255x^2 + 8x + 93 is stored as the uint8 array [255, 8, 93].
        result = bytearray([0] * (degree - 1) + [1])  # Start off with the monomial x^0

        #Compute the product polynomial (x - r^0) * (x - r^1) * (x - r^2) * ... * (x - r^{degree-1}),
        #and drop the highest monomial term which is always 1x^degree.
        #Note that r = 0x02, which is a generator element of this field GF(2^8/0x11D).
        root: int = 1
        for _ in range(degree):  #Unused variable i
            #Multiply the current product by (x - r^i)
            for j in range(degree):
                result[j] = QrCode._reed_solomon_multiply(result[j], root)
                if j + 1 < degree:
                    result[j] ^= result[j + 1]
            root = QrCode._reed_solomon_multiply(root, 0x02)
        return result

    #Returns the Reed-Solomon error correction codeword for the given data and divisor polynomials.
    def _reed_solomon_compute_remainder(data: bytes, divisor: bytes) -> bytes:
        result = bytearray([0] * len(divisor))
        for b in data:  #Polynomial division
            factor: int = b ^ result.pop(0)
            result.append(0)
            for (i, coef) in enumerate(divisor):
                result[i] ^= QrCode._reed_solomon_multiply(coef, factor)
        return result

    #Returns the product of the two given field elements modulo GF(2^8/0x11D).
    def _reed_solomon_multiply(x: int, y: int) -> int:
        if (x >> 8 != 0) or (y >> 8 != 0):
            raise ValueError("Byte out of range")
        #Russian peasant multiplication
        z: int = 0
        for i in reversed(range(8)):
            z = (z << 1) ^ ((z >> 7) * 0x11D)
            z ^= ((y >> i) & 1) * x
        assert z >> 8 == 0
        return z

    def _finder_penalty_count_patterns(self, runhistory: collections.deque) -> int:
        n: int = runhistory[1]
        assert n <= self._size * 3
        core: bool = n > 0 and (runhistory[2] == runhistory[4] == runhistory[5] == n) and runhistory[3] == n * 3
        return (1 if (core and runhistory[0] >= n * 4 and runhistory[6] >= n) else 0) \
             + (1 if (core and runhistory[6] >= n * 4 and runhistory[0] >= n) else 0)


    def _finder_penalty_terminate_and_count(self, currentruncolor: bool, currentrunlength: int, runhistory: collections.deque) -> int:
        if currentruncolor:  # Terminate dark run
            self._finder_penalty_add_history(currentrunlength, runhistory)
            currentrunlength = 0
        currentrunlength += self._size  # Add light border to final run
        self._finder_penalty_add_history(currentrunlength, runhistory)
        return self._finder_penalty_count_patterns(runhistory)


    def _finder_penalty_add_history(self, currentrunlength: int, runhistory: collections.deque) -> None:
        if runhistory[0] == 0:
            currentrunlength += self._size  # Add light border to initial run
        runhistory.appendleft(currentrunlength)

    MIN_VERSION: int =  1  #The minimum version number supported in the QR Code Model 2 standard
    MAX_VERSION: int = 40  #The maximum version number supported in the QR Code Model 2 standard

    #For use in _get_penalty_score(), when evaluating which mask is best.
    _PENALTY_N1: int =  3
    _PENALTY_N2: int =  3
    _PENALTY_N3: int = 40
    _PENALTY_N4: int = 10

    _ECC_CODEWORDS_PER_BLOCK: Sequence[Sequence[int]] = (
        #Version: (note that index 0 is for padding, and is set to an illegal value)
        # 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40    Error correction level
        (-1,  7, 10, 15, 20, 26, 18, 20, 24, 30, 18, 20, 24, 26, 30, 22, 24, 28, 30, 28, 28, 28, 28, 30, 30, 26, 28, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30),  # Low
        (-1, 10, 16, 26, 18, 24, 16, 18, 22, 22, 26, 30, 22, 22, 24, 24, 28, 28, 26, 26, 26, 26, 28, 28, 28, 28, 28, 28, 28, 28, 28, 28, 28, 28, 28, 28, 28, 28, 28, 28, 28),  # Medium
        (-1, 13, 22, 18, 26, 18, 24, 18, 22, 20, 24, 28, 26, 24, 20, 30, 24, 28, 28, 26, 30, 28, 30, 30, 30, 30, 28, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30),  # Quartile
        (-1, 17, 28, 22, 16, 22, 28, 26, 26, 24, 28, 24, 28, 22, 24, 24, 30, 28, 28, 26, 28, 30, 24, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30))  # High

    _NUM_ERROR_CORRECTION_BLOCKS: Sequence[Sequence[int]] = (
        #Version: (note that index 0 is for padding, and is set to an illegal value)
        # 0, 1, 2, 3, 4, 5, 6, 7, 8, 9,10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40    Error correction level
        (-1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 4,  4,  4,  4,  4,  6,  6,  6,  6,  7,  8,  8,  9,  9, 10, 12, 12, 12, 13, 14, 15, 16, 17, 18, 19, 19, 20, 21, 22, 24, 25),  # Low
        (-1, 1, 1, 1, 2, 2, 4, 4, 4, 5, 5,  5,  8,  9,  9, 10, 10, 11, 13, 14, 16, 17, 17, 18, 20, 21, 23, 25, 26, 28, 29, 31, 33, 35, 37, 38, 40, 43, 45, 47, 49),  # Medium
        (-1, 1, 1, 2, 2, 4, 4, 6, 6, 8, 8,  8, 10, 12, 16, 12, 17, 16, 18, 21, 20, 23, 23, 25, 27, 29, 34, 34, 35, 38, 40, 43, 45, 48, 51, 53, 56, 59, 62, 65, 68),  # Quartile
        (-1, 1, 1, 2, 4, 4, 4, 5, 6, 8, 8, 11, 11, 16, 16, 18, 16, 19, 21, 25, 25, 25, 34, 30, 32, 35, 37, 40, 42, 45, 48, 51, 54, 57, 60, 63, 66, 70, 74, 77, 81))  # High

    _MASK_PATTERNS: Sequence[Callable[[int,int],int]] = (
        (lambda x, y:  (x + y) % 2                  ),
        (lambda x, y:  y % 2                        ),
        (lambda x, y:  x % 3                        ),
        (lambda x, y:  (x + y) % 3                  ),
        (lambda x, y:  (x // 3 + y // 2) % 2        ),
        (lambda x, y:  x * y % 2 + x * y % 3        ),
        (lambda x, y:  (x * y % 2 + x * y % 3) % 2  ),
        (lambda x, y:  ((x + y) % 2 + x * y % 3) % 2),
    )

    class Ecc:
        ordinal: int  #(Public) In the range 0 to 3 (unsigned 2-bit integer)
        formatbits: int  #(Package-private) In the range 0 to 3 (unsigned 2-bit integer)

        """The error correction level in a QR Code symbol. Immutable."""
        # Private constructor
        def __init__(self, i: int, fb: int) -> None:
            self.ordinal = i
            self.formatbits = fb

        #Placeholders
        LOW     : QrCode.Ecc
        MEDIUM  : QrCode.Ecc
        QUARTILE: QrCode.Ecc
        HIGH    : QrCode.Ecc

    #Public constants. Create them outside the class.
    Ecc.LOW      = Ecc(0, 1)  # The QR Code can tolerate about  7% erroneous codewords
    Ecc.MEDIUM   = Ecc(1, 0)  # The QR Code can tolerate about 15% erroneous codewords
    Ecc.QUARTILE = Ecc(2, 3)  # The QR Code can tolerate about 25% erroneous codewords
    Ecc.HIGH     = Ecc(3, 2)  # The QR Code can tolerate about 30% erroneous codewords


class QrSegment:
    """A segment of character/binary/control data in a QR Code symbol.
    Instances of this class are immutable.
    This segment class imposes no length restrictions, but QR Codes have restrictions.
    Even in the most favorable conditions, a QR Code can only hold 7089 characters of data.
    Any segment longer than this is meaningless for the purpose of generating QR Codes."""

    #Returns a segment representing the given binary data encoded in byte mode.
    def make_bytes(data: Union[bytes,Sequence[int]]) -> QrSegment:
        bb = _BitBuffer()
        for b in data:
            bb.append_bits(b, 8)
        return QrSegment(QrSegment.Mode.BYTE, len(data), bb)

    #Returns a segment representing the given string of decimal digits encoded in numeric mode.
    def make_numeric(digits: str) -> QrSegment:
        if not QrSegment.is_numeric(digits):
            raise ValueError("String contains non-numeric characters")
        bb = _BitBuffer()
        i: int = 0
        while i < len(digits):  # Consume up to 3 digits per iteration
            n: int = min(len(digits) - i, 3)
            bb.append_bits(int(digits[i : i + n]), n * 3 + 1)
            i += n
        return QrSegment(QrSegment.Mode.NUMERIC, len(digits), bb)

    #Returns a segment representing the given text string encoded in alphanumeric mode.
    def make_alphanumeric(text: str) -> QrSegment:
        if not QrSegment.is_alphanumeric(text):
            raise ValueError("String contains unencodable characters in alphanumeric mode")
        bb = _BitBuffer()
        for i in range(0, len(text) - 1, 2):  # Process groups of 2
            temp: int = QrSegment._ALPHANUMERIC_ENCODING_TABLE[text[i]] * 45
            temp += QrSegment._ALPHANUMERIC_ENCODING_TABLE[text[i + 1]]
            bb.append_bits(temp, 11)
        if len(text) % 2 > 0:  # 1 character remaining
            bb.append_bits(QrSegment._ALPHANUMERIC_ENCODING_TABLE[text[-1]], 6)
        return QrSegment(QrSegment.Mode.ALPHANUMERIC, len(text), bb)


    #Returns a new mutable list of zero or more segments to represent the given Unicode text string.
    def make_segments(text: str) -> List[QrSegment]:
        #Select the most efficient segment encoding automatically
        if text == "":
            return []
        elif QrSegment.is_numeric(text):
            return [QrSegment.make_numeric(text)]
        elif QrSegment.is_alphanumeric(text):
            return [QrSegment.make_alphanumeric(text)]
        else:
            return [QrSegment.make_bytes(text.encode("UTF-8"))]

    #Returns a segment representing an Extended Channel Interpretation (ECI) designator with the given assignment value.
    def make_eci(assignval: int) -> QrSegment:
        bb = _BitBuffer()
        if assignval < 0:
            raise ValueError("ECI assignment value out of range")
        elif assignval < (1 << 7):
            bb.append_bits(assignval, 8)
        elif assignval < (1 << 14):
            bb.append_bits(0b10, 2)
            bb.append_bits(assignval, 14)
        elif assignval < 1000000:
            bb.append_bits(0b110, 3)
            bb.append_bits(assignval, 21)
        else:
            raise ValueError("ECI assignment value out of range")
        return QrSegment(QrSegment.Mode.ECI, 0, bb)


    # Tests whether the given string can be encoded as a segment in numeric mode.
    # A string is encodable iff each character is in the range 0 to 9.
    def is_numeric(text: str) -> bool:
        return QrSegment._NUMERIC_REGEX.fullmatch(text) is not None


    # Tests whether the given string can be encoded as a segment in alphanumeric mode.
    # A string is encodable iff each character is in the following set: 0 to 9, A to Z
    # (uppercase only), space, dollar, percent, asterisk, plus, hyphen, period, slash, colon.
    def is_alphanumeric(text: str) -> bool:
        return QrSegment._ALPHANUMERIC_REGEX.fullmatch(text) is not None

    # The mode indicator of this segment. Accessed through get_mode().
    _mode: QrSegment.Mode

    # The length of this segment's unencoded data. Measured in characters for
    # numeric/alphanumeric/kanji mode, bytes for byte mode, and 0 for ECI mode.
    # Always zero or positive. Not the same as the data's bit length.
    # Accessed through get_num_chars().
    _numchars: int

    # The data bits of this segment. Accessed through get_data().
    _bitdata: List[int]


    # ---- Constructor (low level) ----

    def __init__(self, mode: QrSegment.Mode, numch: int, bitdata: Sequence[int]) -> None:
        """Creates a new QR Code segment with the given attributes and data.
        The character count (numch) must agree with the mode and the bit buffer length,
        but the constraint isn't checked. The given bit buffer is cloned and stored."""
        if numch < 0:
            raise ValueError()
        self._mode = mode
        self._numchars = numch
        self._bitdata = list(bitdata)  # Make defensive copy


    # ---- Accessor methods ----

    def get_mode(self) -> QrSegment.Mode:
        """Returns the mode field of this segment."""
        return self._mode

    def get_num_chars(self) -> int:
        """Returns the character count field of this segment."""
        return self._numchars

    def get_data(self) -> List[int]:
        """Returns a new copy of the data bits of this segment."""
        return list(self._bitdata)  # Make defensive copy


    # Package-private function
    def get_total_bits(segs: Sequence[QrSegment], version: int) -> Optional[int]:
        """Calculates the number of bits needed to encode the given segments at
        the given version. Returns a non-negative number if successful. Otherwise
        returns None if a segment has too many characters to fit its length field."""
        result = 0
        for seg in segs:
            ccbits: int = seg.get_mode().num_char_count_bits(version)
            if seg.get_num_chars() >= (1 << ccbits):
                return None  # The segment's length doesn't fit the field's bit width
            result += 4 + ccbits + len(seg._bitdata)
        return result


    # ---- Constants ----

    # Describes precisely all strings that are encodable in numeric mode.
    _NUMERIC_REGEX: re.Pattern = re.compile(r"[0-9]*")

    # Describes precisely all strings that are encodable in alphanumeric mode.
    _ALPHANUMERIC_REGEX: re.Pattern = re.compile(r"[A-Z0-9 $%*+./:-]*")

    # Dictionary of "0"->0, "A"->10, "$"->37, etc.
    _ALPHANUMERIC_ENCODING_TABLE: Dict[str,int] = {ch: i for (i, ch) in enumerate("0123456789ABCDEFGHIJKLMNOPQRSTUVWXYZ $%*+-./:")}


    # ---- Public helper enumeration ----

    class Mode:
        """Describes how a segment's data bits are interpreted. Immutable."""

        _modebits: int  # The mode indicator bits, which is a uint4 value (range 0 to 15)
        _charcounts: Tuple[int,int,int]  # Number of character count bits for three different version ranges

        # Private constructor
        def __init__(self, modebits: int, charcounts: Tuple[int,int,int]):
            self._modebits = modebits
            self._charcounts = charcounts

        # Package-private method
        def get_mode_bits(self) -> int:
            """Returns an unsigned 4-bit integer value (range 0 to 15) representing the mode indicator bits for this mode object."""
            return self._modebits

        # Package-private method
        def num_char_count_bits(self, ver: int) -> int:
            """Returns the bit width of the character count field for a segment in this mode
            in a QR Code at the given version number. The result is in the range [0, 16]."""
            return self._charcounts[(ver + 7) // 17]

        # Placeholders
        NUMERIC     : QrSegment.Mode
        ALPHANUMERIC: QrSegment.Mode
        BYTE        : QrSegment.Mode
        KANJI       : QrSegment.Mode
        ECI         : QrSegment.Mode

    # Public constants. Create them outside the class.
    Mode.NUMERIC      = Mode(0x1, (10, 12, 14))
    Mode.ALPHANUMERIC = Mode(0x2, ( 9, 11, 13))
    Mode.BYTE         = Mode(0x4, ( 8, 16, 16))
    Mode.KANJI        = Mode(0x8, ( 8, 10, 12))
    Mode.ECI          = Mode(0x7, ( 0,  0,  0))



# ---- Private helper class ----

class _BitBuffer(list):
    """An appendable sequence of bits (0s and 1s). Mainly used by QrSegment."""

    def append_bits(self, val: int, n: int) -> None:
        """Appends the given number of low-order bits of the given
        value to this buffer. Requires n >= 0 and 0 <= val < 2^n."""
        if (n < 0) or (val >> n != 0):
            raise ValueError("Value out of range")
        self.extend(((val >> i) & 1) for i in reversed(range(n)))


    def _get_bit(x: int, i: int) -> bool:
        """Returns true iff the i'th bit of x is set to 1."""
        return (x >> i) & 1 != 0



class DataTooLongError(ValueError):
    """Raised when the supplied data does not fit any QR Code version. Ways to handle this exception include:
    - Decrease the error correction level if it was greater than Ecc.LOW.
    - If the encode_segments() function was called with a maxversion argument, then increase
      it if it was less than QrCode.MAX_VERSION. (This advice does not apply to the other
      factory functions because they search all versions up to QrCode.MAX_VERSION.)
    - Split the text data into better or optimal segments in order to reduce the number of bits required.
    - Change the text or binary data to be shorter.
    - Change the text to fit the character set of a particular segment mode (e.g. alphanumeric).
    - Propagate the error upward to the caller/user."""
    pass

In [52]:
def do_basic_demo() -> None:
    """Creates a single QR Code, then prints it to the console."""
    text = "a"
    errcorlvl = QrCode.Ecc.LOW

    # Make and print the QR Code symbol
    qr = QrCode.encode_text(text, errcorlvl)
    print_qr(qr)

def print_qr(qrcode: QrCode) -> None:
    """Prints the given QrCode object to the console."""
    border = 4
    for y in range(-border, qrcode.get_size() + border):
        for x in range(-border, qrcode.get_size() + border):
            print("\u2588 "[1 if qrcode.get_module(x,y) else 0] * 2, end="")
        print()
    print()

do_basic_demo()

██████████████████████████████████████████████████████████
██████████████████████████████████████████████████████████
██████████████████████████████████████████████████████████
██████████████████████████████████████████████████████████
████████              ████  ████  ██              ████████
████████  ██████████  ██████    ████  ██████████  ████████
████████  ██      ██  ██  ████    ██  ██      ██  ████████
████████  ██      ██  ██    ████████  ██      ██  ████████
████████  ██      ██  ████      ████  ██      ██  ████████
████████  ██████████  ██████      ██  ██████████  ████████
████████              ██  ██  ██  ██              ████████
██████████████████████████        ████████████████████████
██████████████    ██    ██  ██  ██████████    ████████████
████████    ████    ████  ██      ██        ████  ████████
██████████      ██    ██  ██  ████████  ██  ████  ████████
██████████  ████    ██  ██████████        ████  ██████████
████████  ████            ████  ██                ██████

In [53]:
def do_variety_demo() -> None:
    """Creates a variety of QR Codes that exercise different features of the library, and prints each one to the console."""
    #Numeric mode encoding (3.33 bits per digit)
    qr = QrCode.encode_text("314159265358979323846264338327950288419716939937510", QrCode.Ecc.MEDIUM)
    #print_qr(qr)

    #Alphanumeric mode encoding (5.5 bits per character)
    qr = QrCode.encode_text("DOLLAR-AMOUNT:$39.87 PERCENTAGE:100.00% OPERATIONS:+-*/", QrCode.Ecc.HIGH)
    #print_qr(qr)

    # Unicode text as UTF-8
    qr = QrCode.encode_text("\u3053\u3093\u306B\u3061\u0077\u0061\u3001\u4E16\u754C\uFF01\u0020\u03B1\u03B2\u03B3\u03B4", QrCode.Ecc.QUARTILE)
    print_qr(qr)

do_variety_demo()

██████████████████████████████████████████████████████████████████████████████████
██████████████████████████████████████████████████████████████████████████████████
██████████████████████████████████████████████████████████████████████████████████
██████████████████████████████████████████████████████████████████████████████████
████████              ████  ████████  ████  ██        ██████              ████████
████████  ██████████  ██  ████████  ████  ██  ██████████████  ██████████  ████████
████████  ██      ██  ████    ██  ████  ████████    ██  ████  ██      ██  ████████
████████  ██      ██  ██  ██    ████      ██  ██  ██  ██  ██  ██      ██  ████████
████████  ██      ██  ██  ████              ██████      ████  ██      ██  ████████
████████  ██████████  ██████        ██      ██  ████  ██  ██  ██████████  ████████
████████              ██  ██  ██  ██  ██  ██  ██  ██  ██  ██              ████████
████████████████████████  ██████  ████      ██████      ██████████████████████████
████

In [50]:
def do_segment_demo() -> None:
    """Creates QR Codes with manually specified segments for better compactness."""

    # Illustration "silver"
    silver0 = "THE SQUARE ROOT OF 2 IS 1."
    silver1 = "41421356237309504880168872420969807856967187537694807317667973799"
    qr = QrCode.encode_text(silver0 + silver1, QrCode.Ecc.LOW)
    print_qr(qr)

    segs = [
        QrSegment.make_alphanumeric(silver0),
        QrSegment.make_numeric(silver1)]
    qr = QrCode.encode_segments(segs, QrCode.Ecc.LOW)
    print_qr(qr)

    # Illustration "golden"
    golden0 = "Golden ratio \u03C6 = 1."
    golden1 = "6180339887498948482045868343656381177203091798057628621354486227052604628189024497072072041893911374"
    golden2 = "......"
    qr = QrCode.encode_text(golden0 + golden1 + golden2, QrCode.Ecc.LOW)
    #print_qr(qr)

    segs = [
        QrSegment.make_bytes(golden0.encode("UTF-8")),
        QrSegment.make_numeric(golden1),
        QrSegment.make_alphanumeric(golden2)]
    qr = QrCode.encode_segments(segs, QrCode.Ecc.LOW)
    #print_qr(qr)

    # Illustration "Madoka": kanji, kana, Cyrillic, full-width Latin, Greek characters
    madoka = "\u300C\u9B54\u6CD5\u5C11\u5973\u307E\u3069\u304B\u2606\u30DE\u30AE\u30AB\u300D\u3063\u3066\u3001\u3000\u0418\u0410\u0418\u3000\uFF44\uFF45\uFF53\uFF55\u3000\u03BA\u03B1\uFF1F"
    qr = QrCode.encode_text(madoka, QrCode.Ecc.LOW)
    print_qr(qr)

do_segment_demo()

██████████████████████████████████████████████████████████████████████████████████
██████████████████████████████████████████████████████████████████████████████████
██████████████████████████████████████████████████████████████████████████████████
██████████████████████████████████████████████████████████████████████████████████
████████              ████  ████      ██  ████  ██    ██████              ████████
████████  ██████████  ██████        ██    ████  ██  ██  ████  ██████████  ████████
████████  ██      ██  ██      ██    ████    ██████    ██  ██  ██      ██  ████████
████████  ██      ██  ████      ████        ████████    ████  ██      ██  ████████
████████  ██      ██  ██████  ████████████  ████  ██  ██  ██  ██      ██  ████████
████████  ██████████  ████  ██████  ██  ██  ██      ████  ██  ██████████  ████████
████████              ██  ██  ██  ██  ██  ██  ██  ██  ██  ██              ████████
████████████████████████      ██        ████          ██  ████████████████████████
████